# Modelagem de tópicos em textos através do Latent Dirichlet Allocation (LDA) aplicação em python 


## imports 

In [1]:
print('Heloo world')

Heloo world


In [1]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk import wordpunct_tokenize

import re
from time import time

[nltk_data] Downloading package stopwords to /home/renato/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Dados



In [2]:
df = pd.read_csv('../Dados coletados/Twitter_padronizado.csv', encoding='utf-8',  lineterminator='\n')

df.head()

,Unnamed: 0,ID,USUARIO,TWEET,DATA_TWEET,language
0,1,2,meliiuza,RT @joshfeelsz: Meu Deus Eu tô em loop com ess...,2020-09-22 14:17:14,portuguese
1,9,11,Karine77379754,RT @majorjhs: eu nem fico mais surpresa com a ...,2020-09-22 14:17:19,portuguese
2,19,22,EloaSuzi,RT @fabiofaria5555: PR @jairbolsonaro discursa...,2020-09-22 14:17:23,portuguese
3,21,24,Djan00392004,RT @planalto: #AoVivo: Presidente @jairbolsona...,2020-09-22 14:17:22,portuguese
4,22,25,barbaralustoza,biroliro falando de cristofobia na Assembleia ...,2020-09-22 14:17:22,portuguese


## Pré processamento

In [3]:
def remove_elementos_repetidos(lista):
    nova_lista = []
    for item in lista:
        if item not in nova_lista:
            nova_lista.append(item)
    return nova_lista

# remove special characters and digits
def clean_tweet(tweet):
    # remove os RT
    tweet = re.sub(r'RT+', '', tweet) 
    
    # remove as menções
    tweet = re.sub(r'@\S+', '', tweet)  
    
    # remove links e alguns pontos
    tweet = re.sub(r'kkk\S+', '', tweet)
    tweet = re.sub(r"http\S+", "", tweet).lower().replace('.','').replace(';','').replace('-','').replace(':','').replace(')','')
    
    # remove alguns caracteres
    tweet  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",tweet )
    tweet = ' '.join(s for s in tweet.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    tweet = tweet.replace("\n", "")
    return tweet

In [4]:
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'dos'])
clean_tweets = []
for w in range(len(df.TWEET)):
  tweet = df['TWEET'].iloc[w]
  tweet = clean_tweet(tweet)
    
  clean_tweets.append(tweet)

#remover colunas repetidas
clean_tweets = remove_elementos_repetidos(clean_tweets)
clean_tweets[110:145]



#procurar exemplos de limpesa de dados mais eficientes, com tweets

['casagrande não consegue acertar comentários que faz sobre futebol imagine ele comentando política melhor ele proc',
 'mesmo puto com flamengo você vai postar hoje tem flamengo sim',
 'hoje tem flamengo',
 'hoje tem flamengo cobracoral papagaio vintém vestiu rubronegro não tem pra ninguém',
 'hoje tem mengão libea mais querido enfrenta barcelona guayaquil fora casa pela fase grupos',
 'bom dia nação hoje tem flamengo hoje tem mais querido hoje tem maior torcida mundo vamos flamengo',
 'hoje tem flamengooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooooo',
 'gravação golden vamos mais clipe iconico',
 'vídeo marcelinho carioca diz que globo usada por desportistas para atacar bolsonaro vários atletas estamos con',
 'marcelinho carioca calado poeta',
 'querido marcelinho carioca para envergonhar fiel',
 'marcelinho carioca diz que globo usada por desportistas para atacar bolsonaro exjogador marcelinho carioca',
 'marcelinho carioca uma

### LDA funciona baseado em frequências de palavras, então usaremos TFs, e não TF-IDFs.

In [5]:
# COUNT vectorizer
tf_vectorizer = CountVectorizer(
        min_df = 30,
        max_df = 0.5,
        max_features = 10000,
        stop_words = stop_words, 
        ngram_range = (1,2)
  )

#transform
vec_text = tf_vectorizer.fit_transform(clean_tweets)

#returns a list of words.
words = tf_vectorizer.get_feature_names()
#print(clean_tweets)
#print(vec_text)
print(len(words))

4569


Encontrando tópicos
O resultado terá

uma matriz que descreve a relação entre palavras e tópicos
uma matriz que descreve a relação entre documentos e tópicos
Existe uma outra implementação de LDA popular em python chamada Gensim.

In [6]:
from sklearn.decomposition import LatentDirichletAllocation

In [9]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = df[['USUARIO', 'TWEET']].iloc[d]
          print('{} - {} : \t{:.2f}'.format(doc_data[0], doc_data[1], W[d, topic_idx]))

In [11]:
lda = LatentDirichletAllocation(n_components=15, 
                                learning_method='online', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text)

print("done in %0.3fs." % (time() - t0))

done in 119.522s.


In [30]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(86872, 20)
Matriz tópicos-termos:(20, 4573)


In [12]:
display_topics(doc_topic_matrix,
               lda.components_, 
               words,
               df,
               15, 
               10)


--
Topic #1: 
MÊS, QUER, ALGUÉM, MILHÕES, CARALHO, SABER, LUGAR, POVO, TOMAR, PUTA, THE, ONDE, NESSA, FILHO, CERTO
MikaelsonNinan - RT @PatriciaFelixRJ: Alô @jairbolsonaro, seu mentiroso: o povo brasileiro quer saber pra onde foram os mil dólares de auxílio emergencial! : 	0.94
devilofbillie - se chama no time to die mas tá matando todo mundo de ansiedade... enfim a hipocrisia : 	0.88
19Pablo19_ - RT @UOLEsporte: Pay-per-view: você precisa de R$ 3,7 mil/ano para assinar todos de esporte https://t.co/mI6Q3ncWMI : 	0.88
ElianadaCruzRo1 - RT @MariluParreiras: … acrescentou que concedeu auxílio-emergencial de aproximadamente “1 mil dólares para 65 milhões de famílias”. https:/… : 	0.84
Criaonipresente - @VeteranoLouco @yetz1 Já vai ter jogo sendo lançado junto ao console cara sempre lançam jogos junto com os consoles… https://t.co/Kn1pVxJ1KT : 	0.82
prizellei - #DomingoEspetacular oi querida da pra acabar logo pra eu assistir #AFazenda12? : 	0.82
paulaclimaadv - Tá muito #catfish essa nov

yungpeepw - quem é esse peep? peep é uma lenda? nossa vocês sabiam q o peep é uma lenda? pq ele é uma lenda? pq ele é o LIL PEE… https://t.co/oh4tkDzIEi : 	0.87
_cmrgx - RT @vthgbrxv_: as pessoas olhando a foto que o Zé Neto postou : 	0.87

--
Topic #7: 
BRUNO, FINAL, HORA, VOLTA, GUIBI, PARECE, NOME, FAVOR, GUIBI FUTCARLU, FUTCARLU, MARRONE, PRIOR, TCHÊ, TIRAR, JOE
SiegGamer23 - Uma das poucas coisas que eu esperava desse governo bosta, e até hoje nada. Foda que mesmo se acontecer, não vai se… https://t.co/KLCG339SAs : 	0.92
MnicaSalome2 - @Rconstantino Kalil e um canalha!! : 	0.88
Mary97422983 - RT @KellyRiibas: Nem acredito que já fazem oito meses, queria minhas meninas juntxxx 
MABIA DAY : 	0.87
MarcosJosSant16 - RT @Secomstantino: @claudioedantas Estavam o protegendo dele mesmo, claro.

Fácil de se perceber q ao falarem sobre a CPMF usam sempre eufe… : 	0.84
Lica_yoonseok - @softfanx Não kkk, mas eu tenho muita vontade de passear por lá e pretendo fazer isso daqui a uns anos : 	0.

amaduh_oliveira - #HHRL7  ESTA PERFEITO CARA! L7 NÃO ERRA NUNCA! QUE SOM BRABOOOOOOO
#l7nnon : 	0.88
Rmarquess00 - esse barcelona ai é o madureira : 	0.84
dayvidbixa - RT @vittarbz: Aguardando a primeira drag queen brasileira, afeminada, nordestina, com indicação ao grammy participar do #MTVMIAW2020 Pabllo… : 	0.84
Young_Rafa77 - RT @Beto_Morumbi: Vitor Bueno e Pablo são a pior dupla de ataque que já vi, em 39 anos de estádio, no #SPFC 
Não criam, não finalizam, não… : 	0.84
SetiJose - RT @CARLOSNEVESSS: Um recado para você jovem que está entrando no mundo das drogas,pare o quanto antes,depois vai ficar igual o Casagrande… : 	0.84
amendesnovais1 - RT @geglobo: GP da Rússia: Hamilton consegue pole na corrida em que poderá igualar recorde de Michael Schumacher https://t.co/k3Cbnfxxrr ht… : 	0.81
GSantiago14 - olha isso. barcelona é um peso na vida dos cara. suarez continua jogando muito man : 	0.81
noicefriends - @butler_daily eu reclamava de perder pa procurando as pessoas na escola mas

## Visualizando os tópicos

In [21]:
#!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [22]:
pyLDAvis.sklearn.prepare(lda, vec_text, tf_vectorizer, sort_topics=False, mds = 'tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster      Freq
topic                                                   
0      -10.878068   10.538728       1        1  4.071995
1      -60.884026  -75.591820       2        1  4.494647
2       -2.336119  -53.289074       3        1  4.691321
3      -61.385319  -12.914634       4        1  5.430799
4      -28.737743  126.296997       5        1  5.118466
5        1.066606 -123.552811       6        1  4.122639
6       45.140720   -7.947869       7        1  4.681747
7     -140.795975  -98.530548       8        1  4.968323
8       61.531681  -83.240562       9        1  4.240044
9      -72.563057 -140.396881      10        1  4.645090
10      15.984491   63.578186      11        1  4.966332
11    -160.851624   68.945145      12        1  5.125461
12      85.961830   51.922935      13        1  5.268228
13     -99.020836  109.936348      14        1  5.380584
14    -176.764130   -8.681684      15        1  5.175408
15    -117.189880  -37.420200      16        1  4.525312
16     108.829918  -22.946203      17        1  4.990703
17      47.588097  121.934814      18        1  6.634111
18    -106.467834   30.176889      19        1  5.644404
19     -50.613804   59.328842      20        1  5.824385, topic_info=           Term         Freq        Total Category  logprob  loglift
4297        vai  5069.000000  5069.000000  Default  30.0000  30.0000
3831        ser  3922.000000  3922.000000  Default  29.0000  29.0000
4111       time  3059.000000  3059.000000  Default  28.0000  28.0000
1816      gente  2899.000000  2899.000000  Default  27.0000  27.0000
1211        dia  3922.000000  3922.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
632        cair   272.500954   273.443421  Topic20  -4.8876   2.8397
1360  eliminado   259.932554   260.875021  Topic20  -4.9348   2.8395
4329      vamos  1098.693600  1425.219941  Topic20  -3.4934   2.5829
1537    falando   796.592717  1022.024439  Topic20  -3.8149   2.5939
1130     demais   662.984188   918.653781  Topic20  -3.9985   2.5170

[653 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
18        1  0.992925  absurdo
21       19  0.997653   acabar
25       20  0.998054   acabou
34        6  0.997792     acha
40        5  0.997093    achei
...     ...       ...      ...
4548      3  0.998211    álbum
4558     17  0.996523     ódio
4562      4  0.996615   última
4565      7  0.996052   último
4571     11  0.998368    único

[632 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])